# _**PRÁCTICA 5 - MIND**_ 

_Pablo Martín de Benito_

---

In [6]:
import pandas as pd
import numpy as np
import scipy.cluster.hierarchy as sch


from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans,AgglomerativeClustering
from sklearn.model_selection import StratifiedKFold
from statistics import mode
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split

from sklearn_extra.cluster import KMedoids
from sklearn.cluster import DBSCAN

In [7]:
datos = pd.read_table("seeds_dataset.txt", header=None)
datos

,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


Transformamos los datos

In [8]:
X = datos.iloc[:,:-1]



In [9]:
escalado = StandardScaler()
X_scaled = escalado.fit_transform(X)
y = datos.iloc[:,-1]


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=1/3.,stratify = y)

### **Kmedias Encapsulado**

In [11]:
def kmedias(X_train=np.array([]), y_train = np.array([]), X_test =np.array([]), y_test=np.array([]), n_clusters=2, init=None):
    km = KMeans(n_clusters=n_clusters, init=init)
    labels_train = km.fit_predict(X_train)
    moda = np.array([mode(y_train[labels_train == i]) for i in range(n_clusters)])

    if X_test.size == 0:
        accuracy = np.mean(moda[labels_train] == y_train)
    else:
        labels_test = km.predict(X_test)
        accuracy = np.mean(moda[labels_test] == y_test)
    return accuracy

### KMedias con cálculo de centros iniciales

NOTA:  Para elegir el número de clusters, hacemos el número de baricentros iniciales sumado al número de baricentros multiplicado por 1.5
    12*1.5 = 18 + 12 = 30

In [12]:
kmedias_init = KMeans(n_clusters=30)
kmedias_init.fit(X_scaled)
kmedias_init_centros = kmedias_init.cluster_centers_

Los centros del kmeans los utilizamos para el método aglomerativo.

In [13]:
K = 12
aglom = AgglomerativeClustering(n_clusters=K, metric = 'euclidean', linkage = 'ward')
aglom_labels = aglom.fit_predict(kmedias_init_centros)
aglom_centros = np.array([np.mean(kmedias_init_centros[aglom_labels == i], axis = 0) for i in range(12)])
K_init = kmedias_init_centros[np.argmin(pairwise_distances(kmedias_init_centros, aglom_centros), axis = 0)]

No podemos meter el _K_init_ directamente porque son puntos ficticios.

### Kmedias final

X, calcula los puntos más cercanos a los baricentros del ward

In [14]:
X_scaled[np.argmin(pairwise_distances(X_scaled, K_init), axis = 0)].shape

(12, 7)

In [15]:
kmedias(X_train=X_scaled, y_train=y, n_clusters=12, init=X_scaled[np.argmin(pairwise_distances(X_scaled,K_init), axis=0)])

0.9095238095238095

NOTA:  Cogemos los 30 clusters y aplicamos el algoritmo de Ward para seleccionar 12.
Calculas baricentros de todos los clusters y nos quedamos con el argmin por filas.


---

## **KMedoids**

Tenemos que instalar un compilador de C y C++ para poder instalar el extra.



> sudo apt install gcc g++ \
> pip install scikit-learn-extra

In [28]:
kmedoides = KMedoids(n_clusters=12, init='k-medoids++')
labels_kmedoides = kmedoides.fit_predict(X_scaled)
moda_kmedoides = np.array([mode(y[labels_kmedoides == c]) for c in range(12)])
accuracy_kmedoides = np.mean(moda_kmedoides[labels_kmedoides] == y)
accuracy_kmedoides

0.8904761904761904

## **DBScan**

In [17]:
dbscan = DBSCAN(eps=0.6, min_samples=2)
dbscan.fit(X_scaled)
dbscan.labels_

array([ 0, -1,  1,  1,  2,  1,  1,  1, -1, -1, -1,  1, -1,  1,  1, -1, -1,
        2, -1,  3, -1,  1,  2, -1, -1, -1, -1, -1,  1, -1, -1, -1,  4,  1,
        0, -1, -1, -1,  1, -1,  5,  5,  6, -1,  1,  1,  0,  1,  1,  0,  1,
       -1,  4,  1, -1,  0,  1, -1,  0, -1, -1, -1, -1,  3, -1, -1,  7,  1,
        7,  3,  8,  9, 10, 11, 10,  9,  8, -1, -1, -1, -1, -1, 12, 13, 11,
       11, -1, -1, -1, -1, 13, 11, 11, -1, -1,  8, 11, -1, -1, 11, -1, 11,
       11, 11, 11, 11, 11, 10, -1, 11, 11, -1, -1, -1, -1, 11, -1, 11, 11,
       11, 12, 10,  9, 14, -1, 11,  8, 11, 13, -1, 14, 11, -1,  9, -1, -1,
       10, 15, 15, 10, -1, -1, 16,  3,  3, 17, -1,  3,  3, 18,  3, -1,  3,
        3,  3, 17,  3, 19, 19, 17,  3,  3,  3,  3, -1, -1,  3,  3,  3,  3,
       17, -1,  3, 20, 18, -1, 20,  3, 20, -1, -1,  3,  3,  3, 16,  3, 17,
        3, -1, -1, 20,  3, 21,  3,  3,  3,  3, -1,  3,  6,  3,  6, -1, 22,
        3, 21,  3, 22,  3,  3])

In [19]:
np.sum(dbscan.labels_ == -1)

70

### Eliminación de puntos asilados

In [22]:
X_sin_aislados = np.delete(X_scaled, np.argwhere(dbscan.labels_ == -1), axis = 0)
y_sin_aislados = np.delete(y, np.argwhere(dbscan.labels_ == -1), axis = 0)
etiquetas_sin_aislados = np.array(dbscan.labels_[dbscan.labels_ != -1])

In [23]:
np.unique(dbscan.labels_[dbscan.labels_ != -1]).shape[0]

23

In [25]:
moda_dbscan = np.array([mode(y_sin_aislados[etiquetas_sin_aislados == c]) for c in np.unique(etiquetas_sin_aislados)])
moda_dbscan

array([1, 1, 1, 3, 1, 1, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
       3])

In [29]:
accuracy_dbscan = np.mean(moda_dbscan[etiquetas_sin_aislados] == y_sin_aislados)
accuracy_dbscan

0.9714285714285714

Los mejores resultados los obtenemos con el DBScan.